In [58]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [87]:
import os

# Ruta del conjunto de datos
dataset_path = '/content/drive/MyDrive/dataset'

# Verifica la cantidad de imágenes en cada clase
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        images = os.listdir(folder_path)
        print(f"{folder}: {len(images)} imágenes")


Ulises: 130 imágenes
Juan: 130 imágenes
Joel: 130 imágenes


In [76]:
import os
from PIL import Image

# Ruta del conjunto de datos
DATASET_PATH = '/content/drive/MyDrive/dataset'
IMAGE_SIZE = (224, 224)
VALID_EXTENSIONS = ['.jpg', '.jpeg', '.png']  # Formatos permitidos

def check_and_clean_images(dataset_path, image_size=IMAGE_SIZE, valid_extensions=VALID_EXTENSIONS):
    """
    Verifica que las imágenes en el conjunto de datos tengan el formato correcto y las dimensiones adecuadas.
    Recorta las imágenes para asegurarse de que tengan el tamaño adecuado y guarda las imágenes válidas.

    Args:
        dataset_path (str): Ruta al directorio del dataset
        image_size (tuple): Tamaño deseado para las imágenes (altura, ancho)
        valid_extensions (list): Formatos de imagen permitidos

    Returns:
        None
    """
    cleaned_data_path = '/content/drive/MyDrive/cleaned_dataset'

    # Crea la ruta para guardar las imágenes limpias
    if not os.path.exists(cleaned_data_path):
        os.makedirs(cleaned_data_path)

    # Recorre las carpetas de clases
    for class_folder in os.listdir(dataset_path):
        class_folder_path = os.path.join(dataset_path, class_folder)

        if os.path.isdir(class_folder_path):
            cleaned_class_folder = os.path.join(cleaned_data_path, class_folder)
            if not os.path.exists(cleaned_class_folder):
                os.makedirs(cleaned_class_folder)

            # Recorre las imágenes dentro de cada clase
            for image_name in os.listdir(class_folder_path):
                image_path = os.path.join(class_folder_path, image_name)

                # Verifica la extensión de la imagen
                if any(image_name.lower().endswith(ext) for ext in valid_extensions):
                    try:
                        # Abre la imagen
                        with Image.open(image_path) as img:
                            # Recorta o redimensiona la imagen si es necesario
                            img = img.resize(image_size)

                            # Guarda la imagen limpia en el nuevo directorio
                            cleaned_image_path = os.path.join(cleaned_class_folder, image_name)
                            img.save(cleaned_image_path)
                            print(f"Imagen guardada: {cleaned_image_path}")
                    except Exception as e:
                        print(f"Error al procesar la imagen {image_path}: {e}")
                else:
                    print(f"Formato de imagen no permitido: {image_path}")

# Llamada a la función para verificar y limpiar las imágenes
check_and_clean_images(DATASET_PATH)


Imagen guardada: /content/drive/MyDrive/cleaned_dataset/Ulises/WhatsApp Image 2024-12-06 at 2.46.39 PM.jpeg
Imagen guardada: /content/drive/MyDrive/cleaned_dataset/Ulises/WhatsApp Image 2024-12-06 at 2.46.40 PM.jpeg
Imagen guardada: /content/drive/MyDrive/cleaned_dataset/Ulises/WhatsApp Image 2024-12-06 at 2.46.40 PM (1).jpeg
Imagen guardada: /content/drive/MyDrive/cleaned_dataset/Ulises/WhatsApp Image 2024-12-06 at 2.46.40 PM (2).jpeg
Imagen guardada: /content/drive/MyDrive/cleaned_dataset/Ulises/WhatsApp Image 2024-12-06 at 2.46.40 PM (3).jpeg
Imagen guardada: /content/drive/MyDrive/cleaned_dataset/Ulises/WhatsApp Image 2024-12-06 at 2.46.41 PM.jpeg
Imagen guardada: /content/drive/MyDrive/cleaned_dataset/Ulises/WhatsApp Image 2024-12-06 at 2.46.41 PM (1).jpeg
Imagen guardada: /content/drive/MyDrive/cleaned_dataset/Ulises/WhatsApp Image 2024-12-06 at 2.46.41 PM (2).jpeg
Imagen guardada: /content/drive/MyDrive/cleaned_dataset/Ulises/WhatsApp Image 2024-12-06 at 2.46.41 PM (3).jpeg
Imag

In [77]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Configuración de rutas
DATASET_PATH = '/content/drive/MyDrive/cleaned_dataset'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

def prepare_dataset(dataset_path):
    """
    Prepara el dataset para entrenamiento

    Args:
        dataset_path (str): Ruta al directorio del dataset

    Returns:
        tuple: Generadores de datos de entrenamiento y validación
    """
    # Generador de datos con aumento de imagen
    data_generator = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2  # 20% para validación
    )

    # Generador de datos de entrenamiento
    train_generator = data_generator.flow_from_directory(
        dataset_path,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )

    # Generador de datos de validación
    validation_generator = data_generator.flow_from_directory(
        dataset_path,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, validation_generator

# Uso
train_data, val_data = prepare_dataset(DATASET_PATH)
num_classes = len(train_data.class_indices)
print(f"Clases detectadas: {train_data.class_indices}")


Found 312 images belonging to 3 classes.
Found 78 images belonging to 3 classes.
Clases detectadas: {'Joel': 0, 'Juan': 1, 'Ulises': 2}


In [78]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten,
    Dense, Dropout, BatchNormalization, Input
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2

def create_improved_facial_recognition_model(input_shape, num_classes):
    """
    Crea un modelo CNN mejorado para reconocimiento facial usando transfer learning

    Args:
        input_shape (tuple): Dimensiones de la imagen de entrada
        num_classes (int): Número de clases a clasificar

    Returns:
        tf.keras.Model: Modelo CNN compilado
    """
    # Base pre-entrenada
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet',
        pooling='avg'
    )

    # Descongelar más capas del modelo base
    base_model.trainable = True
    fine_tune_at = 100  # Selecciona cuántas capas descongelar
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Modelo secuencial
    model = Sequential([
        # Capa de entrada
        Input(shape=input_shape),

        # Base pre-entrenada
        base_model,

        # Capas personalizadas
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),

        # Capa de salida
        Dense(num_classes, activation='softmax')
    ])

    # Compilación del modelo
    model.compile(
        optimizer=Adam(learning_rate=1e-5),  # Usar una tasa de aprendizaje más baja
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def create_data_augmentation():
    """
    Crear generador de datos con aumento más agresivo

    Returns:
        ImageDataGenerator: Generador de datos aumentados
    """
    return ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,  # Aumentar el rango de rotación
        width_shift_range=0.3,  # Aumentar el desplazamiento horizontal
        height_shift_range=0.3,  # Aumentar el desplazamiento vertical
        shear_range=0.3,  # Aumentar la deformación
        zoom_range=0.3,  # Aumentar el rango de zoom
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest',
        validation_split=0.2
    )

# Configuración de hiperparámetros
INPUT_SHAPE = (224, 224, 3)
NUM_CLASSES = len(train_data.class_indices)

# Crear generador de datos
data_generator = create_data_augmentation()

# Crear modelo
model = create_improved_facial_recognition_model(INPUT_SHAPE, NUM_CLASSES)

# Callbacks para optimización
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,  # Reducir más la tasa de aprendizaje cuando no haya mejoras
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Guardar el mejor modelo
model_checkpoint = ModelCheckpoint(
    'best_facial_recognition_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# Entrenamiento del modelo
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=[reduce_lr, early_stop, model_checkpoint]
)

# Evaluar el modelo
evaluation = model.evaluate(val_data)
print(f"Precisión de validación: {evaluation[1]*100:.2f}%")

# Guardar modelo final
model.save('facial_recognition_model.keras')


Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2878 - loss: 2.2084
Epoch 1: val_accuracy improved from -inf to 0.33333, saving model to best_facial_recognition_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.2914 - loss: 2.1828 - val_accuracy: 0.3333 - val_loss: 1.1448 - learning_rate: 1.0000e-05
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3603 - loss: 1.7832
Epoch 2: val_accuracy improved from 0.33333 to 0.37179, saving model to best_facial_recognition_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.3590 - loss: 1.7844 - val_accuracy: 0.3718 - val_loss: 1.1289 - learning_rate: 1.0000e-05
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4980 - loss: 1.3254
Epoch 3: val_accuracy did not improve from 0.37179
10/10 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.5008 - loss: 1.3206 - val_accuracy: 0.3718 - val_loss: 1.0832 - learning_rate: 1.0000e-05
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accurac

In [86]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(model, image_path, threshold=0.6):
    """
    Realiza la predicción de una imagen con el modelo entrenado y devuelve la clase predicha solo si la probabilidad es alta.

    Args:
        model (tf.keras.Model): El modelo entrenado
        image_path (str): Ruta a la imagen que se quiere predecir
        threshold (float): Umbral de probabilidad para considerar una predicción válida

    Returns:
        tuple: Clase predicha y su probabilidad, o None si no alcanza el umbral
    """
    # Cargar la imagen y preprocesarla
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expandir la dimensión para que sea un batch de tamaño 1
    img_array = img_array / 255.0  # Normalizar

    # Realizar la predicción
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    predicted_prob = predictions[0][predicted_class[0]]  # Obtener la probabilidad de la clase predicha

    # Verificar si la probabilidad es mayor que el umbral
    if predicted_prob < threshold:
        return None, None  # No hay predicción confiable

    # Obtener el nombre de la clase predicha
    predicted_class_name = list(train_data.class_indices.keys())[predicted_class[0]]

    return predicted_class_name, predicted_prob

# Ejemplo de predicción
img_path = '/content/drive/MyDrive/prueba/gato.jpg'
predicted_class, predicted_prob = predict_image(model, img_path)

if predicted_class is None:
    print("La imagen no pertenece a ninguna de las clases conocidas.")
else:
    print(f"Predicción: {predicted_class} con probabilidad de {predicted_prob*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicción: Juan con probabilidad de 60.57%


In [84]:
import tensorflow as tf

# Cargar el modelo entrenado
saved_model_path = 'best_facial_recognition_model.keras'
model = tf.keras.models.load_model(saved_model_path)

# Convertir el modelo a TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Guardar el modelo convertido en formato .tflite
tflite_model_path = 'model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Modelo convertido a TensorFlow Lite y guardado en: {tflite_model_path}")


Saved artifact at '/tmp/tmpf9poaemp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_23')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  134498142140048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134497323820560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134498345774848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134497976089264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134497323815456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134498417129984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134497327257152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134497327255568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134498141748944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134497327263664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134497327262